## 2024 -25 Roster Explore Notebook #1
#### Created 8/25/24 -by JBS

Scraping code taken from Roster_scraper_variable.ipynb in the original college_hockey folder

In [18]:
import os
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re


# print list of folders one dir up from current
# print(os.listdir(os.path.join(os.getcwd(), '..')))

# path ro TEMP folder
temp_folder = os.path.join(os.getcwd(), '..', 'TEMP')
# Data folder
data_folder = os.path.join(os.getcwd(), '..', 'data')
# Image folder
img_folder = os.path.join(os.getcwd(), '..', 'images')

# # save file into each folder for testing

# file = os.path.join(temp_folder, 'test.txt')
# with open(file, 'w') as f:
#     f.write('test')

### Get All Scheduled games for the given season in CHN Data

In [19]:
# Set the URL to scrape
url_base = 'https://www.collegehockeynews.com/schedules/?season=' # Link to the 2022-2023 season with all results

#### THESE NEED TO BE UPDATED EACH SEASON ####
#### Set the season varriable portion of the url 
url_season = '20242025'
#### string to add to file names and whatnote
file_tag = '2024_current_sept_24'

############# Set the output folder to the TEMP folder for testing
final_output_dir = os.path.join(temp_folder)

# # Set the output folder to the data folder
# final_output_dir = os.path.join(data_folder, 'schedule')


# temp_output_dir = '../TEMP/results/'



#########################################################################


## Construction the url to feel to parser
url = url_base + url_season

# Get the page with requests
response = requests.get(url)

# Create a BeautifulSoup object
soup = BeautifulSoup(response.text, 'html.parser')

# select the table or tables
tables = soup.find_all('table')

# Initialize variables
current_date = None
current_conference = None
game_notes = None

# Initialize an empty list to hold the data
data = []

# Parse the table with BeautifulSoup

rows = soup.find_all('tr')

# Loop through each row to find relevant information
for row in rows:
    # Check for date row
    if row.get('class') == ['stats-section']:
        current_date = row.find('td').text.strip()
    # Check for conference row
    elif row.get('class') == ['sked-header']:
        current_conference = row.find('td').text.strip()
    # Check for game notes
    elif len(row.find_all('td')) == 2:
        game_notes = row.find_all('td')[1].text.strip()
    # Process rows with game data
    elif row.get('valign') == 'top':
        cells = row.find_all('td')
        if len(cells) >= 9:
            home_team = cells[0].text.strip()
            home_team_link = cells[0].find('a')['href'] if cells[0].find('a') else None
            home_score = cells[1].text.strip()
            away_team = cells[3].text.strip()
            away_team_link = cells[3].find('a')['href'] if cells[3].find('a') else None
            away_score = cells[4].text.strip()
            ot = cells[5].text.strip()
            box_link = cells[7].find('a')['href'] if cells[7].find('a') else None
            metrics_link = cells[8].find('a')['href'] if cells[8].find('a') else None
             # Capture Game Notes
            game_notes_cell = cells[-1].find('small')
            game_notes = game_notes_cell.text.strip() if game_notes_cell else None

            # Append data to the list
            data.append([current_date, current_conference, game_notes, home_team, home_team_link, home_score, away_team, away_team_link, away_score, ot, box_link, metrics_link])
            game_notes = None  # Reset game notes for the next row
            

# Create a DataFrame
columns = ['Date', 'Conference', 'Game_Notes', 'Away_Team', 'Away_Team_Link', 'Away_Score', 'Home_Team', 'Home_Team_Link', 'Home_Score',  'OT', 'Box_Link', 'Metrics_Link']
df = pd.DataFrame(data, columns=columns)




### Some Data Tranformation
#### Creates Unique GAME_ID
- Cleaning string formats, 
- create team abbv

In [20]:
## Extract the day of the week from the date and save in new column
df['Day'] = pd.to_datetime(df['Date']).dt.day_name()
# remove day of the week from date
# format data column as YYYY-MM-DD
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

### Create a new column for the game ID
## Game ID will be a combination of the date and abbreviated team names

# Function to abbreviate the team names
for row in df.itertuples():
    home_team = row.Home_Team
    away_team = row.Away_Team
    home_team_abbr = home_team.split(' ')[-1]
    away_team_abbr = away_team.split(' ')[-1]
    game_id = f'{row.Date}_{home_team_abbr}_{away_team_abbr}'
    df.loc[row.Index, 'Game_ID'] = game_id

# Create a new column for the game ID
df['Game_ID'] = df['Game_ID'].str.replace(',', '')

# Apply the function to the DataFrame
df['Game_ID'] = df.apply(lambda row: f'{row.Date}_{row.Home_Team}_{row.Away_Team}', axis=1)

# STORE AS SCHED_DF
sched_df = df.copy()

# Save the DataFrame to a CSV file
# IN THE TEMP FOLDER
sched_df.to_csv(os.path.join(final_output_dir, f'schedule_{file_tag}.csv'), index=False)
# TO THE DATA FOLDER
# sched_df.to_csv(os.path.join(data_folder, f'schedule_{file_tag}.csv'), index=False)



In [21]:
# sched_df.head()

## Get The Roster Data

In [22]:

## Load results for season to get team links
# Read the CSV file
# results = pd.read_csv('../../data/results_table_all_time.csv')
results = sched_df.copy()

season = 2024

# # Concatenate the unique values in 'Home_Team_Link' and 'Away_Team_Link'
team_links = pd.concat([results['Home_Team_Link'], results['Away_Team_Link']])

# Drop duplicates and NaN values
team_links = team_links.drop_duplicates().dropna()

# Clean up the links - drop everything before the third slash - keep the team name and the school ID as string
team = team_links.str.split('/', expand=True)[3]
# ORIGINAL
number = team_links.str.split('/', expand=True)[4]

# NEW - Create a string for the year '{season}{season+1}'
year = str(season) + str(season+1)
# reconstruct the link and store in a new column
team_links = 'https://www.collegehockeynews.com/reports/roster/' + team + '/' + number + '/' + year

# reset the index
team_links = team_links.reset_index(drop=True)






team_links


# Output results
print(f'Number of team links: {len(team_links)}')
print(f'TEST LINK: FIRST IN LIST: {team_links[0]}')
print(f'TEST LINK: LAST IN LIST: {team_links.iloc[-1]}')


Number of team links: 64
TEST LINK: FIRST IN LIST: https://www.collegehockeynews.com/reports/roster/Lake-Superior/24/20242025
TEST LINK: LAST IN LIST: https://www.collegehockeynews.com/reports/roster/Long-Island/62/20242025


## Functions that clean the roster dataframes

In [23]:
df.head()# Function to split "Last Team" into "Team" and "League" with edge case handling
def split_last_team(last_team):
    # Use regular expression to extract team and league
    match = re.search(r'(.+) \((.+)\)', last_team)
    if match:
        return match.groups()
    else:
        # If no league is specified, return the team as is and leave league blank
        return last_team, ""

# Updated function to correctly capture the player's position and handle edge cases in "Last Team"
def parse_and_transform_roster(html_content):
    # Initialize BeautifulSoup object
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find the table with the roster
    roster_table = soup.find('table', {'id': 'players'})

    # Check if the table exists
    if roster_table is None:
        print("Skipped: The table doesn't exist.")
        return None
    
    # Extract headers
    header_row = roster_table.find('thead').find('tr')
    headers = [header.text.strip() for header in header_row.find_all('th')]
    headers.append('Position')  # Add the Position column to headers
    
    # Initialize data list and current_position variable
    data = []
    current_position = None  # Initialize as None to later filter out irrelevant rows
    
    # Iterate through each row in the table
    for row in roster_table.find_all('tr'):
        if 'class' in row.attrs and 'stats-section' in row.attrs['class']:
            current_position = row.text.strip()
        else:
            cells = row.find_all('td')
            if cells and current_position:
                row_data = [cell.text.strip() for cell in cells]
                row_data.append(current_position)  # Add the current position to the row data
                data.append(row_data)
    
    # Create DataFrame
    df = pd.DataFrame(data, columns=headers)

    # Check if DataFrame is empty or if key columns are missing
    if df.empty or 'Last Team' not in df.columns or 'NHL Draft' not in df.columns:
        print("Skipped: The DataFrame is empty or missing key columns.")
        return None
    
    # Cleanup: Remove rows where 'No.' column is not numeric
    df = df[df['No.'].str.isnumeric()]
    
    # Cleanup: Drop the 'Pos' column
    df.drop(columns=['Pos'], inplace=True)
    
    # Transform Height to Inches
    df['Height_Inches'] = df['Ht.'].apply(convert_to_inches)

    # Transform NHL Draft to Draft_Year, NHL_Team, and D_Round
    # Transform NHL Draft to Draft_Year, NHL_Team, and D_Round
    draft_result = df['NHL Draft'].apply(split_nhl_draft)

    # Check if there are enough values to unpack
    if len(draft_result) > 0:
        df['Draft_Year'], df['NHL_Team'], df['D_Round'] = zip(*draft_result)
    else:
        # Handle the case when result is empty
        df['Draft_Year'], df['NHL_Team'], df['D_Round'] = [None] * len(df), [None] * len(df), [None] * len(df)

    df.drop(columns=['NHL Draft'], inplace=True) # Drop the original NHL Draft column
    
    # Handle edge cases in "Last Team" to split into "Team" and "League"
    df['Team'], df['League'] = zip(*df['Last Team'].apply(split_last_team))
    df.drop(columns=['Last Team'], inplace=True)
    
    # Rename the trouble column Hometown\nLast Team\nNHL Draft
    df.rename(columns={'Hometown\nLast Team\nNHL Draft': 'Hometown'}, inplace=True)

    # assign data types No. Wt. and Height_Inches to int, DOB to datetime
    int_list = ['No.', 'Wt.', 'Height_Inches']
    # Convert columns to numeric, coercing errors to NaN
    df[int_list] = df[int_list].apply(pd.to_numeric, errors='coerce')

    # Replace NaNs with a default value for specific columns
    default_value = 0
    df[int_list] = df[int_list].fillna(value=default_value)

    # Confirm that NaNs are filled
    print(df[int_list].isna().sum())  # Should output all zeros

    # Convert the columns to integers
    df[int_list] = df[int_list].astype(int)

    # df = df[df['Height_Inches'].notna()]  # Assuming convert_to_inches returns None for bad values

    
    return df

# Function to split "NHL Draft" into "Draft_Year", "NHL_Team", and "D_Round"
def split_nhl_draft(nhl_draft):
    try:
        draft_year, nhl_team, d_round = nhl_draft.split('-')
        return draft_year, nhl_team, d_round
    except ValueError:
        # Handle missing or incomplete data
        return None, None, None

# # Test the function
# test_values = ['2022-WSH-7', '', '2021-DET']
# [split_nhl_draft(val) for val in test_values]



# Function to convert height in "ft-in" format to total inches
def convert_to_inches(height_str):
    try:
        feet, inches = map(int, height_str.split('-'))
        return (feet * 12) + inches
    except ValueError:
        return None

### Run the roster scrape and parser for the list of team name

In [24]:
# Correcting the code to handle the 'Team' and 'Last Team' columns properly

## Loop through the team links and parse the roster data 
### Notes: save the roster link to the dataframe and add the team name and school ID

roster_dfs = []  # Assuming this list exists to store each roster DataFrame

# Extract team names from team_links
team_names = pd.Series(team_links).str.split('/', expand=True)[5]


for i, link in enumerate(team_links):
    print(f'Processing team {i+1} of {len(team_links)}')
    
    try:
        # Make GET request to team link
        r = requests.get(link)
        r.raise_for_status()  # This will raise an HTTPError if the HTTP request returned an unsuccessful status code
        html_content = r.text
        
        # Parse and transform the roster data
        roster_df = parse_and_transform_roster(html_content)
    
    except requests.RequestException as e:
        print(f"Error during the request for team {i+1} ({team_names.iloc[i]}): {e}")
        continue
    
    except Exception as e:
        print(f"Error processing the data for team {i+1} ({team_names.iloc[i]}): {e}")
        continue
    
    # Check if the DataFrame exists (i.e., the page had content)
    if roster_df is None:
        print(f"Skipping team {i+1} due to missing or empty data.")
        continue  # Skip this iteration and move to the next one

    try:
        # Reset the index if it's not unique
        roster_df.reset_index(drop=True, inplace=True)
        
        # Add the team name
        current_team = team_names.iloc[i]  # Extract the current team name
        roster_df['Current Team'] = current_team  # Add it to the DataFrame

        # Add a column with the season
        roster_df['Season'] = season
        
        # Add the roster DataFrame to the list
        roster_dfs.append(roster_df)
    
    except Exception as e:
        print(f"Error working with the DataFrame for team {i+1} ({team_names.iloc[i]}): {e}")
        continue

# Assuming the last dataframe processed is the one of interest (can be adjusted later)
# roster_df = roster_dfs[-1]

# Add all the DataFrames in the list to a single DataFrame
roster_df = pd.concat(roster_dfs, ignore_index=True)

Processing team 1 of 64


### Further Roster Cleaning Steps


In [8]:
## OUTPUT TO TEMP FOLDER FOR EXAMINATION
# roster_df.to_csv(os.path.join(temp_folder, f'roster_{file_tag}.csv'), index=False)

In [9]:


# Further transformations

roster_df['Current Team'] = roster_df['Current Team'].str.replace('-', ' ') # Remove - [dash] from Current Team
roster_df.columns = roster_df.columns.str.replace('.', '')  # Remove periods in column names
# Split 'Name' into 'First Name' and 'Last Name'
roster_df['Last_Name'] = roster_df['Name'].str.split(',').str[0]  # Extract 'Last Name'
roster_df['First_Name'] = roster_df['Name'].str.split(',').str[1]  # Extract 'First Name'
# roster_df['Name'] = roster_df['Name'].str.split(',').str[::-1].str.join(' ')  # Convert 'Last, First' to 'First Last'
roster_df['Player'] = roster_df['Name']  # Store the Player name as 'First Last'
roster_df['Player'] = roster_df['Player'].str.replace(u'\xa0', u' ').str.strip()  # Cleanup player name
roster_df = roster_df.drop(['Name'], axis=1)  # Drop the original 'Name' column

# Renaming 'Team' to 'Last Team' if it exists in the DataFrame
if 'Team' in roster_df.columns:
    roster_df.rename(columns={'Team': 'Last Team'}, inplace=True)

# Checking if 'Last Team' exists in the dataframe before reordering
if 'Last Team' not in roster_df.columns:
    print("'Last Team' column not found in the DataFrame. Please check the data extraction process.")
else:
    # Reorder columns without duplicate 'Team'
    roster_df = roster_df[['Current Team', 'Last_Name', 'First_Name', 'No', 'Position', 'Yr', 'Ht', 'Wt', 'DOB', 
                           'Hometown', 'Height_Inches', 'Draft_Year', 'NHL_Team', 'D_Round', 'Last Team', 'League']]

roster_df.head() if 'Last Team' in roster_df.columns else None

,Current Team,Last_Name,First_Name,No,Position,Yr,Ht,Wt,DOB,Hometown,Height_Inches,Draft_Year,NHL_Team,D_Round,Last Team,League
0,Lake Superior,Barone,Adam,6,Defensemen,Fr,6-1,174,5/6/2004,"Sault Ste. Marie, Ont.",73,None,None,None,Trail,BCHL
1,Lake Superior,Blanchett,Jack,16,Defensemen,So,5-11,185,5/12/2003,"Monroe, Mich.",71,None,None,None,Powell,BCHL
2,Lake Superior,Brown,Mike,3,Defensemen,Jr,6-2,209,4/3/2001,"Belmont, Mass.",74,None,None,None,Merrimack,
3,Lake Superior,Bushy,Evan,5,Defensemen,So,6-1,195,3/26/2002,"Mankato, Minn.",73,None,None,None,Trail,BCHL
4,Lake Superior,Conrad,Jacob,4,Defensemen,Fr,5-11,180,5/18/2002,"Green Bay, Wis.",71,None,None,None,Fairbanks,NAHL


### Transform the Hometown column
- Explode Hometown into City, State/Provence, Country

In [10]:
# Ensure the Hometown column is a string and handle NaN values
roster_df['Hometown'] = roster_df['Hometown'].astype(str)

# First step: split the Hometown column by commas into two parts (city and state/province)
# We'll explicitly handle cases where there may not be two parts
split_columns = roster_df['Hometown'].str.split(',', n=1, expand=True)

# Assign split values to new columns and handle missing values
roster_df['City'] = split_columns[0].str.strip()
roster_df['State_Province'] = split_columns[1].str.strip() if split_columns.shape[1] > 1 else ''


# Define a list of known USA states and Canadian provinces abbreviations
usa_states = ['Ala.', 'Alaska', 'Ariz.', 'Ark.', 'Calif.', 'Colo.', 'Conn.', 'Del.', 'D.C.', 'DC', 'Fla.', 'Ga.', 'Hawaii', 
              'Idaho', 'Ill.', 'Ind.', 'Iowa', 'Kan.', 'Ky.', 'La.', 'Maine', 'Md.', 'Mass.', 'Mich.', 'Minn.', 'Miss.', 
              'Mo.', 'Mont.', 'Neb.', 'Nev.', 'N.H.', 'N.J.', 'N.M.', 'N.Y.', 'N.C.', 'N.D.', 'Ohio', 'Okla.', 'Ore.', 
              'Pa.', 'R.I.', 'S.C.', 'S.D.', 'Tenn.', 'Texas', 'Utah', 'Vt.', 'Va.', 'Wash.', 'W.Va.', 'Wis.', 'Wisc.', 'Wyo.']

canada_provinces = ['Alta.', 'Alb.', 'B.C.', 'Man.', 'MB', 'N.B.', 'NB', 'N.L.', 'Newf.', 'NWT', 'N.S.', 'N.W.T.', 'Nunavut', 'Ont.', 'P.E.I.', 'Que.', 'Sask.', 'Yukon', 'YT']

# Function to infer country more robustly
def robust_infer_country(state_province):
    if pd.isna(state_province) or state_province == '':
        return None
    state_province = state_province.strip()
    
    # USA state
    if state_province in usa_states:
        return 'USA'
    
    # Canadian province
    if state_province in canada_provinces:
        return 'Canada'
    
    # If it doesn't match USA or Canada, treat the state_province as a country
    return state_province

# Apply the updated function to infer the country for each player
roster_df['Country'] = roster_df['State_Province'].apply(robust_infer_country)

# Country abreviations that need to be replaced ITA-Italy, AUT-Austria, SUI-Switzerland
roster_df['Country'] = roster_df['Country'].replace({'ITA': 'Italy', 'AUT': 'Austria', 'SUI': 'Switzerland'})


In [11]:
### ADDED 9/18/2024

# Need to furthur standardize some state, provence and country names
## Dictionary to standardize state/province names

standardized_locations = {
    'Ont.': 'Ontario', 'Mich.': 'Michigan', 'Mass.': 'Massachusetts', 'Minn.': 'Minnesota', 
    'Wis.': 'Wisconsin', 'Sweden': 'Sweden', 'Germany': 'Germany', 'B.C.': 'British Columbia',
    'N.Y.': 'New York', 'Wash.': 'Washington', 'Que.': 'Quebec', 'Alb.': 'Alberta', 
    'N.J.': 'New Jersey', 'Sask.': 'Saskatchewan', 'Conn.': 'Connecticut', 'Mo.': 'Missouri',
    'Texas': 'Texas', 'Calif.': 'California', 'DC': 'District of Columbia', 'Fla.': 'Florida',
    'Ohio': 'Ohio', 'Ill.': 'Illinois', 'Pa.': 'Pennsylvania', 'Ga.': 'Georgia',
    'Mont.': 'Montana', 'Tenn.': 'Tennessee', 'Colo.': 'Colorado', 'Va.': 'Virginia', 
    'Vt.': 'Vermont', 'R.I.': 'Rhode Island', 'Md.': 'Maryland', 'Ariz.': 'Arizona', 
    'Wisc.': 'Wisconsin', 'Iowa': 'Iowa', 'Man.': 'Manitoba', 'Slovakia': 'Slovakia', 
    'N.D.': 'North Dakota', 'N.C.': 'North Carolina', 'P.E.I.': 'Prince Edward Island',
    'N.H.': 'New Hampshire', 'Alaska': 'Alaska', 'Belarus': 'Belarus', 'MB': 'Manitoba',
    'Russia': 'Russia', 'Finland': 'Finland', 'Newf.': 'Newfoundland and Labrador', 
    'Hungary': 'Hungary', 'SUI': 'Switzerland', 'S.C.': 'South Carolina', 'Latvia': 'Latvia',
    'Czech Republic': 'Czech Republic', 'N.B.': 'New Brunswick', 'Great Britain': 'United Kingdom', 
    'NB': 'New Brunswick', 'Norway': 'Norway', 'N.S.': 'Nova Scotia', 'Ind.': 'Indiana', 
    'NWT': 'Northwest Territories', 'AUT': 'Austria', 'Idaho': 'Idaho', 'S.D.': 'South Dakota', 
    'Switzerland': 'Switzerland', 'Ore.': 'Oregon', 'Wyo.': 'Wyoming', 'Utah': 'Utah', 
    'ITA': 'Italy', 'Slovenia': 'Slovenia', 'YT': 'Yukon', 'Del.': 'Delaware', 'Maine': 'Maine',
    'Poland': 'Poland', 'Yukon': 'Yukon', 'Ukraine': 'Ukraine', 'Japan': 'Japan', 'Neb.': 'Nebraska'
}

## Apply the standardization to the State/Province column
roster_df['State_Province'] = roster_df['State_Province'].replace(standardized_locations)


## OUTPUT CLEANED ROSTER FILE

In [12]:
### CHECK RESULTING DATAFRAME
roster_df.head()

roster_df.info()

roster_df.sample(20)

# Store todays data as a string
date = pd.to_datetime('today').strftime('%Y%m%d')

# Save the DataFrame to a CSV file
# In the TEMP folder
roster_df.to_csv(os.path.join(final_output_dir, f'roster_{file_tag}v3_ex{date}.csv'), index=False)
# In the DATA folder
roster_df.to_csv(os.path.join(data_folder, f'roster_{file_tag}_v3_ex{date}.csv'), index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1819 entries, 0 to 1818
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Current Team    1819 non-null   object
 1   Last_Name       1819 non-null   object
 2   First_Name      1819 non-null   object
 3   No              1819 non-null   int32 
 4   Position        1819 non-null   object
 5   Yr              1819 non-null   object
 6   Ht              1819 non-null   object
 7   Wt              1819 non-null   int32 
 8   DOB             1819 non-null   object
 9   Hometown        1819 non-null   object
 10  Height_Inches   1819 non-null   int32 
 11  Draft_Year      224 non-null    object
 12  NHL_Team        224 non-null    object
 13  D_Round         224 non-null    object
 14  Last Team       1819 non-null   object
 15  League          1819 non-null   object
 16  City            1819 non-null   object
 17  State_Province  1819 non-null   object
 18  Country 

In [13]:
# Number of players with no Last Team and or League vs total number of players
missing_team = roster_df['Last Team'].isna().sum()
missing_league = roster_df['League'].isna().sum()

print(f'Number of players with missing Last Team: {missing_team}')
print(f'Number of players with missing League: {missing_league}')
print(f'Total number of players: {len(roster_df)}')
print('########################################\n')
print('########################################')
print(f'Distribution of players by position:\n{roster_df["Position"].value_counts()}')
print('########################################\n')



Number of players with missing Last Team: 0
Number of players with missing League: 0
Total number of players: 1819
########################################

########################################
Distribution of players by position:
Position
Forwards       1033
Defensemen      585
Goaltenders     201
Name: count, dtype: int64
########################################



In [14]:
## Teams with the most draft picks
# Filter out players with missing draft information
draft_df = roster_df[roster_df['Draft_Year'].notna()]

# Group by 'Current Team' and count the number of draft picks
team_draft_counts = draft_df['Current Team'].value_counts()
print(f'Teams with the most draft picks:\n{team_draft_counts.head(20)}')

Teams with the most draft picks:
Current Team
Minnesota            16
North Dakota         13
Boston University    13
Denver               13
Harvard              12
Boston College       12
Providence           11
Minnesota Duluth     10
Massachusetts         9
Michigan              8
Colorado College      8
Western Michigan      8
Northeastern          7
Wisconsin             7
Michigan State        7
Cornell               6
St Cloud State        6
Notre Dame            6
Omaha                 5
Ohio State            5
Name: count, dtype: int64


In [15]:
# Player by League
print('########################################')
print(f'Distribution of players by League:\n{roster_df["League"].value_counts()}')

## Distrobution by class year
print('########################################\n')
print(f'Distribution of players by Class Year:\n{roster_df["Yr"].value_counts()}')



########################################
Distribution of players by League:
League
USHL              550
NAHL              341
BCHL              283
AJHL               82
HEA                72
                   58
NCHC               55
CCHA               44
ECAC               42
NTDP               40
B10                33
AHA                33
OJHL               28
NCDC               28
SJHL               18
Independents       16
MJHL               16
D-I Ind.           13
CCHL               10
J20 Nationell      10
SM-sarja            6
NCAA D3             3
PREP                3
SWE                 3
ACHA                2
AJHL/BCHL           2
NA3HL               2
EHL                 2
USHS                2
D-III               1
PHC                 1
USPHL NCDC          1
Sweden              1
ACHA-II             1
MJAHL               1
USHS-Prep           1
SHL                 1
MHL                 1
DIII                1
DI Independent      1
Midget AAA          1
Prep           

In [16]:
## Value Count of First Name
print('########################################\n')
print(f'Distribution of players by First Name:\n{roster_df["First_Name"].value_counts()}')

## Value Count of Last Name
print('########################################\n')
print(f'Distribution of players by Last Name:\n{roster_df["Last_Name"].value_counts()}')


########################################

Distribution of players by First Name:
First_Name
Ryan       48
Jack       46
Nick       31
Tyler      29
Michael    27
           ..
Gavyn       1
Verner      1
Daimon      1
Karl        1
Tyriq       1
Name: count, Length: 604, dtype: int64
########################################

Distribution of players by Last Name:
Last_Name
Smith          10
Johnson         7
Brown           6
Miller          6
Scott           6
               ..
Hillier         1
Fitzpatrick     1
Daneault        1
Cranston        1
Rupprecht       1
Name: count, Length: 1576, dtype: int64


In [17]:
## Value Count of City
print('########################################\n')
print(f'Distribution of players by Hometown:\n{roster_df["City"].value_counts()}')

## Value Count of State_Province
print('########################################\n')
print(f'Distribution of players by State_Province:\n{roster_df["State_Province"].value_counts()}')

## Value Count of Country
print('########################################\n')
print(f'Distribution of players by Country:\n{roster_df["Country"].value_counts()}')



########################################

Distribution of players by Hometown:
City
Calgary              48
Toronto              27
Winnipeg             16
Plymouth             15
North Vancouver      14
                     ..
Westport              1
Sewell                1
Castle Pines          1
Pendleton             1
Thief River Falls     1
Name: count, Length: 909, dtype: int64
########################################

Distribution of players by State_Province:
State_Province
Minnesota                241
Ontario                  183
Massachusetts            120
Michigan                 117
Alberta                  114
                        ... 
Northwest Territories      1
Prince Edward Island       1
Montana                    1
South Carolina             1
Nebraska                   1
Name: count, Length: 69, dtype: int64
########################################

Distribution of players by Country:
Country
USA               1129
Canada             523
Sweden              57
F